In [ ]:
# !pip install bigqueue-client --upgrade

In [ ]:
# !pip show bigqueue-client

In [1]:
from bigqueue.client.client import BigqueueClient, BigqueueClientConfig# !pip install bigqueue-client --upgrade
from bigqueue.client.entity import *

/home/default_user/.conda/envs/user/lib/python3.10/site-packages/bighead/core/util/conversions.py:36: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  PYTHON_SCALAR_TYPES.add(np.bool)


In [2]:
# STAGING:     'bigqueue_service_url': 'https://bigqueue-staging.a.musta.ch/',
config = BigqueueClientConfig(
    bigqueue_service_url='https://bigqueue-staging.a.musta.ch',
    bigqueue_service_version='0',
    datastore_type='S3',
    datastore_init_kwargs={
        'bucket': 'airbnb-bigqueue',
        'base_prefix': 'staging'
    }
)

In [3]:
config = BigqueueClientConfig(
    bigqueue_service_url='https://bigqueue.a.musta.ch',
    bigqueue_service_version='0',
    datastore_type='S3',
    datastore_init_kwargs={
        'bucket': 'airbnb-bigqueue',
        'base_prefix': 'production'
    }
)

In [4]:
bigq_client = BigqueueClient(config)

In [5]:
model = bigq_client.upload_model(
    owner="shaowei_su",
    env="devel/shaowei/rl:0.0.5",
    model_file_root='/home/tianbing_xu/repos/GRPO/TinyZero',
    model_file_path='verl/trainer/main_srl.py',
    name='Demo Ray Job Codebase for MATH RL',
    flavor='ray')

In [ ]:
job = bigq_client.create_ray_job(
    model_id=model['id'],
    name="SRL GSM (off, kl): Qwen2.5-1.5B",
    owner='tianbing_xu',
    team='one_airbnb_team_id_2934',
    queue='csp-mle-p4',
    execution_config={
        'head': {'cpu': 12, 'memory': '20Gi',  'ephemeral_storage': '50Gi',},
        'worker': {'number': 1, 'gpu': 4, 'accelerator': 'nvidia-tesla-a100xl', 'cpu': 24, 'memory': '300Gi',  'ephemeral_storage': '200Gi',}
    },
    runtime_env={"working_dir": "./", "env_vars": {"WANDB_API_KEY": "09f781a401d2ec1d3ff5f9b550eb1ec0f291fb31", "HF_ENDPOINT": "http://bigqueue-service-production.bigqueue-service-production:6580/hfproxy"}},
    entrypoint="export WANDB_API_KEY=\"09f781a401d2ec1d3ff5f9b550eb1ec0f291fb31\" && export N_GPUS=4 && export BASE_MODEL=Qwen/Qwen2.5-1.5B && export DATA_DIR=./data/gsm8k && export ROLLOUT_TP_SIZE=2 && export EXPERIMENT_NAME=srl_gsm_qwen_15b_off_kl && export VLLM_ATTENTION_BACKEND=XFORMERS && python3 -m verl.trainer.main_srl algorithm.adv_estimator=reinforce data.train_files=$DATA_DIR/train.parquet data.val_files=$DATA_DIR/test.parquet data.train_batch_size=32 data.val_batch_size=32 data.max_prompt_length=512 data.max_response_length=1024 data.filter_overlong_prompts=True actor_rollout_ref.model.path=$BASE_MODEL actor_rollout_ref.actor.optim.lr=1e-6 actor_rollout_ref.model.use_remove_padding=True actor_rollout_ref.actor.ppo_mini_batch_size=32 actor_rollout_ref.actor.ppo_micro_batch_size=4 actor_rollout_ref.actor.use_kl_loss=True actor_rollout_ref.actor.kl_loss_coef=0.001 actor_rollout_ref.actor.use_importance_weight=False actor_rollout_ref.actor.off_policy_pg=True actor_rollout_ref.actor.clip_ratio=0.0 actor_rollout_ref.actor.kl_loss_type=low_var_kl actor_rollout_ref.model.enable_gradient_checkpointing=True actor_rollout_ref.actor.fsdp_config.param_offload=False actor_rollout_ref.actor.fsdp_config.optimizer_offload=False actor_rollout_ref.rollout.log_prob_micro_batch_size=4 actor_rollout_ref.rollout.tensor_model_parallel_size=$ROLLOUT_TP_SIZE actor_rollout_ref.rollout.name=vllm actor_rollout_ref.rollout.gpu_memory_utilization=0.4 actor_rollout_ref.rollout.n=2 actor_rollout_ref.ref.log_prob_micro_batch_size=4 actor_rollout_ref.ref.fsdp_config.param_offload=True algorithm.kl_ctrl.kl_coef=0.01 trainer.logger=\"['console','wandb']\" trainer.project_name=SimpleRL trainer.experiment_name=$EXPERIMENT_NAME trainer.default_hdfs_dir=null trainer.n_gpus_per_node=$N_GPUS trainer.nnodes=1 trainer.save_freq=-1 trainer.test_freq=5 trainer.total_epochs=1",
)

In [9]:
job

{'id': 'eca77a38-7697-427e-aa6d-98f33d1bba63'}